In [7]:
import numpy as np

In [41]:
data = np.loadtxt('x28.txt', skiprows=72)
data

array([[  1.   ,  36.   ,  27.   , ...,  59.   ,  59.   , 921.87 ],
       [  2.   ,  35.   ,  23.   , ...,  39.   ,  57.   , 997.875],
       [  3.   ,  44.   ,  29.   , ...,  33.   ,  54.   , 962.354],
       ...,
       [ 58.   ,  45.   ,  24.   , ...,   8.   ,  56.   , 895.696],
       [ 59.   ,  42.   ,  83.   , ...,  49.   ,  54.   , 911.817],
       [ 60.   ,  38.   ,  28.   , ...,  39.   ,  58.   , 954.442]])

In [58]:
X = data[:,1:-1]
y = data[:,-1]
X,y

(array([[3.600e+01, 2.700e+01, 7.100e+01, 8.100e+00, 3.340e+00, 1.140e+01,
         8.150e+01, 3.243e+03, 8.800e+00, 4.260e+01, 1.170e+01, 2.100e+01,
         1.500e+01, 5.900e+01, 5.900e+01],
        [3.500e+01, 2.300e+01, 7.200e+01, 1.110e+01, 3.140e+00, 1.100e+01,
         7.880e+01, 4.281e+03, 3.600e+00, 5.070e+01, 1.440e+01, 8.000e+00,
         1.000e+01, 3.900e+01, 5.700e+01],
        [4.400e+01, 2.900e+01, 7.400e+01, 1.040e+01, 3.210e+00, 9.800e+00,
         8.160e+01, 4.260e+03, 8.000e-01, 3.940e+01, 1.240e+01, 6.000e+00,
         6.000e+00, 3.300e+01, 5.400e+01],
        [4.700e+01, 4.500e+01, 7.900e+01, 6.500e+00, 3.410e+00, 1.110e+01,
         7.750e+01, 3.125e+03, 2.710e+01, 5.020e+01, 2.060e+01, 1.800e+01,
         8.000e+00, 2.400e+01, 5.600e+01],
        [4.300e+01, 3.500e+01, 7.700e+01, 7.600e+00, 3.440e+00, 9.600e+00,
         8.460e+01, 6.441e+03, 2.440e+01, 4.370e+01, 1.430e+01, 4.300e+01,
         3.800e+01, 2.060e+02, 5.500e+01],
        [5.300e+01, 4.500e+01, 8.00

In [80]:
def normalize_and_add_ones(X):
    X_max = X.max(axis=0)
    X_min = X.min(axis=0)
    
    X_normalized = (X-X_min) / (X_max - X_min)
    ones = np.ones((X.shape[0],1))
    
    return np.hstack((ones, X_normalized))

In [81]:
normalize_and_add_ones(X)


array([[1.        , 0.52      , 0.21126761, 0.36363636, 0.40322581,
        0.68852459, 0.72727273, 0.61506276, 0.21821264, 0.21220159,
        0.33976834, 0.13529412, 0.0309119 , 0.04402516, 0.20938628,
        0.6       ],
       [1.        , 0.5       , 0.15492958, 0.40909091, 0.88709677,
        0.36065574, 0.60606061, 0.50209205, 0.34390894, 0.07427056,
        0.65250965, 0.29411765, 0.01081917, 0.02830189, 0.13718412,
        0.54285714],
       [1.        , 0.68      , 0.23943662, 0.5       , 0.77419355,
        0.47540984, 0.24242424, 0.61924686, 0.34136595, 0.        ,
        0.21621622, 0.17647059, 0.00772798, 0.01572327, 0.11552347,
        0.45714286],
       [1.        , 0.74      , 0.46478873, 0.72727273, 0.14516129,
        0.80327869, 0.63636364, 0.44769874, 0.20392347, 0.69761273,
        0.63320463, 0.65882353, 0.02627512, 0.02201258, 0.08303249,
        0.51428571],
       [1.        , 0.66      , 0.32394366, 0.63636364, 0.32258065,
        0.85245902, 0.18181818, 

In [112]:
class RidgeRegression:
    def __init__(self):
        return
    
    def fit(self, X_train, y_train, LAMDA):
        assert len(X_train.shape) == 2 and \
            X_train.shape[0] == y_train.shape[0]
        
        w = np.linalg.inv(X_train.T.dot(X_train) +
            LAMDA*np.eye(X_train.shape[1])).dot(X_train.T).dot(y_train)
        
        return w
    
    def fit_grad(self, X_train, y_train, LAMDA, lr=0.01, epoch=100,batch_size=32):
        W = np.random.randn(X_train.shape[1])
        last_loss = 10e+8
        for ep in range(epoch):
            arr = np.array(range(X_train.shape[0]))
            np.random.shuffle(arr)
            X_train=X_train[arr]
            y_train=y_train[arr]
            for i in range(int(X_train.shape[0]/batch_size)+1):
                index = i*batch_size
                if i < int(X_train.shape[0]/batch_size):
                    X_train_sub = X_train[index:index+batch_size]
                    y_train_sub = y_train[index:index+batch_size]
                else:
                    X_train_sub = X_train[index:]
                    y_train_sub = y_train[index:]
                grad = X_train_sub.T.dot(X_train_sub.dot(W) - y_train_sub) + LAMDA*W
                W = W - lr*grad
            new_loss = self.compute_RSS(self.predict(W,X_train),y_train)
            if (np.abs(new_loss - last_loss) <= 1e-5):
                break
            last_loss = new_loss
        return W
    
    def predict(self, W, X_test):
        y_pred = X_test.dot(W)
        
        return y_pred
    
    def compute_RSS(self, y_test, y_pred):
        loss = (1/y_test[0])*np.sum((y_test-y_pred)**2)
        
        return loss
    
    def get_LAMDA(self, X_train, y_train):
        def cross_validation(num_folds, LAMDA):
            row_ids = np.array(range(X_train.shape[0]))
            
            valid_ids = np.split(row_ids[:len(row_ids)-len(row_ids) % num_folds], num_folds)
            valid_ids[-1] = np.append(valid_ids[-1], row_ids[len(row_ids) - len(row_ids)%num_folds:])
            train_ids = [[k for k in row_ids if k not in valid_ids[i]] for i in range(num_folds)]
            aver_RSS = 0
            for i in range(num_folds):
                valid_part = {'X': X_train[valid_ids[i]], 'y': y_train[valid_ids[i]]}
                train_part = {'X': X_train[train_ids[i]], 'y': y_train[train_ids[i]]}
                W = self.fit(train_part['X'], train_part['y'], LAMDA)
                y_pred = self.predict(W, valid_part['X'])
                aver_RSS += self.compute_RSS(valid_part['y'], y_pred)
            return aver_RSS / num_folds
        
        def range_scan(best_LAMDA, min_RSS, LAMDA_values):
            for current_LAMDA in LAMDA_values:
                aver_RSS = cross_validation(num_folds=5, LAMDA=current_LAMDA)
                if aver_RSS < min_RSS:
                    best_LAMDA = current_LAMDA
                    min_RSS = aver_RSS
            return best_LAMDA, min_RSS
        
        best_LAMDA, min_RSS = range_scan(best_LAMDA=0,min_RSS=10000**2,LAMDA_values = range(1,50))
        
        LAMDA_values = np.linspace(best_LAMDA-0.5,best_LAMDA+0.5,200)
        
        best_LAMDA, min_RSS = range_scan(best_LAMDA=best_LAMDA,min_RSS=min_RSS,LAMDA_values = LAMDA_values)
        
        return best_LAMDA
    

In [114]:
X_train, y_train = X[:70], y[:70]
X_test, y_test = X[30:], y[30:]

ridge_res = RidgeRegression()
best_LAMDA = ridge_res.get_LAMDA(X_train, y_train)
print('Best LAMDA: ', best_LAMDA)

W_learned = ridge_res.fit(X_train, y_train, best_LAMDA)
y_pred = ridge_res.predict(W_learned, X_test)

print('RSS: ',ridge_res.compute_RSS(y_test, y_pred))

Best LAMDA:  0.5
RSS:  40.54796660481085


In [87]:
a = np.array([1,2,3,4,5,6,7,8,9])
a[id]

array([2, 3, 4, 7, 8])